In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('energydata_complete.csv')

In [43]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [44]:
df_original = df.copy()

In [45]:
df.drop(['date', 'lights'], axis = 1, inplace=True)

In [46]:
df.columns

Index(['Appliances', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
       'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [47]:
from sklearn.preprocessing import MinMaxScaler

In [48]:
scaler = MinMaxScaler()

In [49]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [50]:
target = normalised_df[ 'Appliances' ]

In [35]:
from sklearn.model_selection import train_test_split

In [69]:
x_train, x_test, y_train, y_test = train_test_split(normalised_df, target, test_size= 0.3 , random_state= 42 )

In [70]:
from sklearn.linear_model import LinearRegression

In [71]:
linear_model = LinearRegression()

In [72]:
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [123]:
# fitting a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6).

In [99]:
new_T2 = pd.DataFrame(df['T2'])

In [100]:
new_T6 = pd.DataFrame(df['T6'])

In [101]:
linear_model.fit(new_T2, new_T6)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
correlation_matrix = np.corrcoef(x_values, y_values)

correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2

In [73]:
predicted_values = linear_model.predict(x_test)

In [74]:
#MAE (Practice)
from sklearn.metrics import mean_absolute_error 

In [75]:
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3)

0.0

In [105]:
# Mean Absolute Error (Quiz)
mae = mean_absolute_error(new_T2, new_T6)
round(mae, 2)

12.44

In [76]:
import numpy as np
#RSS
rss = np.sum(np.square(y_test - predicted_values))

In [108]:
#RSS (Quiz)
rss = np.sum(np.square(new_T2 - new_T6))
round(rss, 2)

T2    0.0
T6    0.0
dtype: float64

In [107]:
round(rss, 2)

T2    0.0
T6    0.0
dtype: float64

In [110]:
# Mean Square Error
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(new_T2, new_T6))
round(rmse, 3)

13.229

In [79]:
# R^2 Score (Practice)
from sklearn.metrics import r2_score 
r2_score = r2_score(y_test, predicted_values) 
round(r2_score, 3) 

1.0

In [104]:
# R^2 Score (Quiz)
from sklearn.metrics import r2_score 
r2_score = r2_score(new_T2, new_T6) 
round(r2_score, 2) 

-35.39

In [115]:
from sklearn.linear_model import Ridge 
ridge_reg = Ridge(alpha= 0.5 ) 
ridge_reg.fit(new_T2, new_T6) 

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [116]:
from sklearn.linear_model import Lasso 
lasso_reg = Lasso(alpha= 0.001 ) 
lasso_reg.fit(new_T2, new_T6)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [121]:
def get_weights_df(lasso_reg, feat, col_name) : 
    #this function returns the weight of every feature
    weights = pd.Series(lasso_reg.coef_, feat.columns).sort_values() 
    weights_df = pd.DataFrame(weights).reset_index() 
    weights_df.columns = ['new_T2', col_name] 
    weights_df[col_name].round( 3 ) 
    return weights_df

In [122]:
linear_model_weights = get_weights_df(lasso_reg, new_T2, 'Linear_Model_Weight' ) 
ridge_weights_df = get_weights_df(ridge_reg, new_T2, 'Ridge_Weight' ) 
lasso_weights_df = get_weights_df(lasso_reg, new_T2, 'Lasso_weight' )

Exception: Data must be 1-dimensional

In [91]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on= 'normalise_df' ) 
final_weights = pd.merge(final_weights, lasso_weights_df, on= 'normalise_df' )

In [93]:
final_weights

,normalise_df,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,T7,0.000000,0.000045,0.000000
1,Tdewpoint,0.000000,0.000348,0.000000
2,Visibility,0.000000,0.000055,0.000000
3,Windspeed,0.000000,0.000138,0.000000
4,RH_out,-0.000000,-0.000229,-0.000000
5,Press_mm_hg,-0.000000,0.000030,-0.000000
6,T_out,0.000000,-0.001137,0.000000
7,RH_9,-0.000000,-0.000189,-0.000000
8,T9,0.000000,-0.000855,0.000000
9,RH_8,-0.000000,-0.000710,-0.000000
